In [1]:
import pandas as pd
# pd.set_option('display.max_rows', 300)
import rapidfuzz
from joblib import Parallel, delayed
from tqdm import tqdm
import re
import awswrangler as wr
import boto3
import numpy as np
import os

In [2]:

wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

labelDf = wr.s3.read_parquet(
    path=f"s3://partlist_label/",
    columns=["CaseID", "lvl_1_desc", "lvl_2_desc", "lvl_3_desc" ,"Description",	"fVAL",	"DamageCond"],
)
labelDf

,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
0,12657878,bonnet,bonnet,bonnet,bonnet,550.00,bent/dented
1,12657888,bonnet,bonnet,bonnet,bonnet,510.44,bent
2,12657890,bonnet,bonnet,bonnet,bonnet,504.94,dented
3,12657900,bonnet,bonnet,bonnet,bonnet,450.00,crumpled
4,12657903,bonnet,bonnet,bonnet,bonnet,280.00,bent
...,...,...,...,...,...,...,...
6114698,12606104,rear panel inner trim,rear panel,rear panel,rear panel inner trim,235.00,broken
6114699,12606233,rear panel inner trim,rear panel,rear panel,rear panel inner trim,350.00,dented
6114700,12606480,rear panel inner trim,rear panel,rear panel,rear panel inner trim,250.00,broken
6114701,12606779,rear panel inner trim,rear panel,rear panel,firewall rear trim,225.00,


In [3]:
tempDf = labelDf[labelDf["CaseID"] == 10053736]
tempDf

,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
130318,10053736,bonnet,bonnet,bonnet,bonnet,282.85,
422199,10053736,bumper front cover,bumper front,bumper front,bumper front cover,222.98,
1100382,10053736,headlamp assy rh,headlamp rh,headlamp rh,headlamp assy rh,373.74,
1677745,10053736,grille front,grille,grille,grille front,131.90,
3349762,10053736,fender front rh,fender front rh,fender front rh,fender front rh,100.80,


In [4]:
id = 12

In [5]:
caseStudyDf = pd.read_csv(f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_{id}/case_study_gt.csv")
caseStudyCaseId = caseStudyDf["CaseID"].tolist()


In [6]:
labelDf = labelDf[labelDf["CaseID"].isin(caseStudyCaseId)]
labelDf.sort_values(by=["CaseID", "lvl_3_desc"], inplace=True)
labelDf

/tmp/ipykernel_285882/4094135370.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labelDf.sort_values(by=["CaseID", "lvl_3_desc"], inplace=True)


,CaseID,lvl_1_desc,lvl_2_desc,lvl_3_desc,Description,fVAL,DamageCond
2339956,10047111,none,misc,misc,headlight,5752.90,
2029794,10049081,bumper rear cover,bumper rear,bumper rear,bumper rear cover,5800.00,warped
2029795,10049081,bumper rear cover,bumper rear,bumper rear,bumper rear cover,1570.21,warped
5144032,10049081,rear bumper,bumper rear,bumper rear,bumper reflector rear rh,107.67,broken
4474598,10049081,exhaust manifold cover,engine,engine,rear exhaust heat shield,702.10,crumpled
...,...,...,...,...,...,...,...
1019333,13025161,headlamp assy rh,headlamp rh,headlamp rh,"headlamp assy, rh",1388.80,chipped
2288549,13025161,none,misc,misc,"tyre, b/stone torenza t005 (205/60r17)",250.00,chipped
2902239,13025161,shock absorber front lh,undercarriage,non external,"absorber assy, shock, fr lh",233.20,bent
2954401,13025161,suspension arm front lh,undercarriage,non external,"arm s/a, suspension lwr lh w/bush",242.70,bent


In [7]:
outputDir = f"/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_{id}/partlist"
os.makedirs(outputDir, exist_ok=True)

In [8]:
import plotly.graph_objects as go
for caseId in labelDf["CaseID"].unique().tolist():
    df = labelDf[labelDf["CaseID"] == caseId]
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),                line_color='darkslategray',
),
        cells=dict(values=[df[x] for x in df.columns],                line_color='darkslategray',
)
    )])

    fig.update_layout(width=1200, height=( np.clip(
        
            (len(df) // 4)* 300, 
            a_min=400, a_max=3000
            )
            )  
            )

# fig.show()
    fig.write_image(f"{outputDir}/{caseId}.png") 
